## PAD

Bu notebook streamlit ile mukayese içindir

## Input Data

In [4]:
CONFIG_DATA = "original" #"original",example"
#----------------------------------------
import pandas as pd
import paper as pa
import standard_data as sdata
if CONFIG_DATA == "original":
    TRESHOLD = 0
    df = pd.read_csv("data/t25_completolda_documenti.csv") #orgdata
    df = df.drop(columns=['volume','issue','autori','chiave','year']) #orgdata
elif CONFIG_DATA == "example":
    TRESHOLD = 92
    df = sdata.input_set
else:
    print("CONFIG_DATA should be original or example")
df.head(5).style.set_caption("<b style='background-color: yellow;'>Table 0: Input Data</b>")

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,title
0,0.000000,0.023508,0.165675,0.016695,0.000000,0.000000,0.000000,0.000000,0.205028,0.141577,0.000000,0.000000,0.000000,0.000000,0.279718,0.000000,0.000000,0.000000,0.000000,0.000000,0.167656,0.000000,0.000000,0.000000,0.000000,Stagnation theory and stagnation policy
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019269,0.980557,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Economic reform and income distribution in Hungary
2,0.000000,0.028883,0.000000,0.000000,0.000000,0.933444,0.000000,0.000000,0.000000,0.032239,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Foreign trade and industrial development: Greece and the EEC
3,0.000000,0.000000,0.000000,0.359661,0.000000,0.221246,0.000000,0.000000,0.018811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.306384,0.000000,0.000000,0.000000,0.093746,Origins of the post-war payments system
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.193294,0.000000,0.745569,0.000000,0.000000,0.036501,0.000000,0.000000,0.024540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"Notes on consumption, investment and effective demand: II: PART II: MONETARY ANALYSIS"


## Example Extraction

In [5]:
reat = pa.topics_that_are_sufficient_to_reach_the_threshold(df,rownum = 1, treshold=TRESHOLD)
reat['as_df']

,topic,weight,cumsum
0,t16,0.980557,0.980557


`topics_that_are_sufficient_to_reach_the_threshold` fonksiyonu ne yapıyor: Önce girdi tablosundaki değerleri (p2 	80 	2 	8 	10) alıyor sonra sayılara göre sıralıyor t konuları da ona göre sıralanıyor.  Sonra kümülatif değerleri alıyor

## Table 1: Analysis for each paper

In [6]:
TRESHOLD

0

In [7]:
# columns
notta = 'number of topics that are sufficient to reach the threshold'
lotta = 'label of topics that are sufficient to reach the threshold'

for index,row in df.iterrows():
    reat = pa.topics_that_are_sufficient_to_reach_the_threshold(df,rownum = index,treshold=TRESHOLD)
    df.at[index,notta] = reat['as_count'] # yeni kolona notta adını ver.
    df.at[index,lotta] = reat['as_topic_labels']
    #max min (TODO:Look table 4, maxweight is 40 and min weight 25)
    df.at[index,"max weight"] = max(reat['as_df'].weight) #düz weight değeri en yüksek olanı al
    df.at[index,"min weight"] = min(reat['as_df'].weight)

In [8]:
# styling
df[notta] = df[notta].astype(int)
table_1= df
table_1.head(4)

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,...,t21,t22,t23,t24,t25,title,number of topics that are sufficient to reach the threshold,label of topics that are sufficient to reach the threshold,max weight,min weight
0,0.0,0.023508,0.165675,0.016695,0.0,0.000000,0.0,0.0,0.205028,0.141577,...,0.167656,0.0,0.0,0.0,0.000000,Stagnation theory and stagnation policy,1,t15,0.279718,0.279718
1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,Economic reform and income distribution in Hun...,1,t16,0.980557,0.980557
2,0.0,0.028883,0.000000,0.000000,0.0,0.933444,0.0,0.0,0.000000,0.032239,...,0.000000,0.0,0.0,0.0,0.000000,Foreign trade and industrial development: Gree...,1,t6,0.933444,0.933444
3,0.0,0.000000,0.000000,0.359661,0.0,0.221246,0.0,0.0,0.018811,0.000000,...,0.306384,0.0,0.0,0.0,0.093746,Origins of the post-war payments system,1,t4,0.359661,0.359661


Ne yaptık: `topics_that_are_sufficient_to_reach_the_threshold` fonksiyonu alınıp buraddaki t listesi `label of topics that are sufficient to reach the threshold` kolonuna yazdırılıyor. Sonra bu kolonda geçen konu sayısı `number of topics that are sufficient to reach the threshold` kolonuna yazdırılıyor. Burada mesela 4 demek o makalede 4 konu geçmiş demek, 3 demek 3 konu geçmiş demek. Max ve min weight kolonları ise normal weight'lerden alınıyor

# Statstical Summary for Table 1

In [9]:
df[notta].describe().to_frame().transpose()

,count,mean,std,min,25%,50%,75%,max
number of topics that are sufficient to reach the threshold,1608.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


## Table 2: Summary of the number of topics (Old Reat Table)

In [10]:
t_reat_freq = table_1[notta].value_counts().to_frame().fillna(0).reset_index()
t_reat_perc = round(table_1[notta].value_counts(normalize = True)*100,2).to_frame().reset_index()
t_reat_dist = t_reat_freq.merge(t_reat_perc,on="index")
t_reat_dist.columns = ['topic','reat_freq','reat_perc']
t_reat_dist['cms'] = t_reat_dist.reat_perc.cumsum()
t_reat_dist.columns = ['number of topics that are sufficient to reach the threshold','number of paper','percentage','cumulative percentage']
t_reat_dist.sort_values(by = 'number of topics that are sufficient to reach the threshold')

,number of topics that are sufficient to reach the threshold,number of paper,percentage,cumulative percentage
0,1,1608,100.0,100.0


Ne yaptık: Burada ise Table 1'deki `number of topics that are sufficient to reach the threshold` kolonunun frekans değerleri alındı. Yani içinde 3 konu geçen 3 makale, 4 konu geçen 1 makale var. Kalan kolonlar da buna göre hesaplandı

In [11]:
bottom = t_reat_dist.sum().to_frame().transpose()
bottom['number of topics that are sufficient to reach the threshold']=""
bottom['cumulative percentage'] = ""
bottom

,number of topics that are sufficient to reach the threshold,number of paper,percentage,cumulative percentage
0,,1608.0,100.0,


In [12]:
t_read_dist_with_summary = pd.concat([t_reat_dist,bottom])
t_read_dist_with_summary
t_read_dist_with_summary['number of paper'] = t_read_dist_with_summary['number of paper'].astype("int")
t_read_dist_with_summary

# TODO: number of topics that are sufficient to reach the threshold kolonu sıralı olacak.

,number of topics that are sufficient to reach the threshold,number of paper,percentage,cumulative percentage
0,1,1608,100.0,100.0
0,,1608,100.0,


## Table 3: Summary of the topic distribution (Old  Suffi Table)

In [13]:
def get_order(topicno):
    return topicno.split("t")[1]

In [14]:
TRESHOLD

0

In [15]:
number_of_the_paper_in_the_corpus = df.count()[0]
number_of_the_paper_in_the_corpus
# TODO: Alessandra's request: Keep this number in a proper place in the application

1608

In [16]:
melted_topics = pd.DataFrame(",".join(table_1[lotta].to_list()).split(","))
tfreq = melted_topics[0].value_counts().to_frame().reset_index()
tfreq.columns = ['topic','number of paper']
count_of_papers_in_corpus = table_1.count()[0]
number_of_papers_in_corpus = tfreq['number of paper']/count_of_papers_in_corpus
tfreq['percentage of papers in the corpus'] = (number_of_papers_in_corpus*100).round(2)
tfreq['order'] = tfreq['topic'].apply(get_order).astype(int)
tfreq = tfreq.sort_values(by = "order")
tfreq = tfreq[['topic','number of paper','percentage of papers in the corpus']]
tfreq
# TODO: When you deploy ANY COLUMN should be sortable by the USER

,topic,number of paper,percentage of papers in the corpus
11,t1,73,4.54
15,t2,50,3.11
18,t3,37,2.30
4,t4,98,6.09
12,t5,72,4.48
7,t6,88,5.47
1,t7,112,6.97
24,t8,11,0.68
0,t9,132,8.21
2,t10,107,6.65


Ne yaptı: Burada Tablo 1'de geçen makale sayısı `count_of_papers_in_corpus` ve her bir konunun table1'de kaç kez geçtiğini kullanarak bu tabloyu oluşturduk.

# Table 4: Maximum and mininum weights table

In [17]:
bins_example = [0,10,20,30,40,50,60,70,80,90,100]
bins_original = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
#rule
# start from zero
# end from one
bins = ""
if CONFIG_DATA == "original":
    bins = bins_original
elif CONFIG_DATA == "example":
    bins = bins_example
else:
    print ("Error")

tmaxwei_freq = pd.cut(table_1['max weight'],bins = bins).value_counts(sort = False).to_frame().fillna(0).reset_index()
tminwei_freq = pd.cut(table_1['min weight'],bins = bins).value_counts(sort = False).to_frame().fillna(0).reset_index()
tmaxminwei_freq = tmaxwei_freq.merge(tminwei_freq)
tmaxminwei_freq.columns = ["bin","pa_max","pa_min"]
pa_max_sum = tmaxminwei_freq.pa_max.sum()
pa_min_sum = tmaxminwei_freq.pa_min.sum()
#tmaxminwei_freq.append(tmaxminwei_freq.sum(numeric_only=True), ignore_index=True)

tmaxminwei_freq['perc_max'] = tmaxminwei_freq.pa_max / pa_max_sum
tmaxminwei_freq['perc_min'] = tmaxminwei_freq.pa_min / pa_min_sum

tmaxminwei_freq['cumsum_max'] = tmaxminwei_freq.perc_max.cumsum()
tmaxminwei_freq['cumsum_min'] = tmaxminwei_freq.perc_min.cumsum()
tmaxminwei_freq = tmaxminwei_freq[['bin','pa_max','perc_max','cumsum_max','pa_min','perc_min','cumsum_min']]
#tmaxminwei_freq.append(tmaxminwei_freq.sum(numeric_only=True), ignore_index=True) # summary table

tmaxminwei_freq.loc["Row_Total"] = tmaxminwei_freq.sum()
tmaxminwei_freq

,bin,pa_max,perc_max,cumsum_max,pa_min,perc_min,cumsum_min
0,"(0.0, 0.1]",0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,"(0.1, 0.2]",2.0,0.001244,0.001244,2.0,0.001244,0.001244
2,"(0.2, 0.3]",66.0,0.041045,0.042289,66.0,0.041045,0.042289
3,"(0.3, 0.4]",206.0,0.128109,0.170398,206.0,0.128109,0.170398
4,"(0.4, 0.5]",300.0,0.186567,0.356965,300.0,0.186567,0.356965
5,"(0.5, 0.6]",291.0,0.180970,0.537935,291.0,0.180970,0.537935
6,"(0.6, 0.7]",229.0,0.142413,0.680348,229.0,0.142413,0.680348
7,"(0.7, 0.8]",214.0,0.133085,0.813433,214.0,0.133085,0.813433
8,"(0.8, 0.9]",151.0,0.093905,0.907338,151.0,0.093905,0.907338
9,"(0.9, 1.0]",149.0,0.092662,1.000000,149.0,0.092662,1.000000


# Table 5: Maximum and mininum weights table for subset

In [18]:
USER_INPUT_T = 2

In [19]:
subset_table = table_1[table_1['number of topics that are sufficient to reach the threshold'] == USER_INPUT_T]

In [20]:
tmaxwei_freq = pd.cut(subset_table['max weight'],bins = bins).value_counts(sort = False).to_frame().fillna(0).reset_index()
tminwei_freq = pd.cut(subset_table['min weight'],bins = bins).value_counts(sort = False).to_frame().fillna(0).reset_index()
tmaxminwei_freq = tmaxwei_freq.merge(tminwei_freq)
tmaxminwei_freq.columns = ["bin","pa_max","pa_min"]
pa_max_sum = tmaxminwei_freq.pa_max.sum()
pa_min_sum = tmaxminwei_freq.pa_min.sum()
#tmaxminwei_freq.append(tmaxminwei_freq.sum(numeric_only=True), ignore_index=True)

tmaxminwei_freq['perc_max'] = tmaxminwei_freq.pa_max / pa_max_sum
tmaxminwei_freq['perc_min'] = tmaxminwei_freq.pa_min / pa_min_sum

tmaxminwei_freq['cumsum_max'] = tmaxminwei_freq.perc_max.cumsum()
tmaxminwei_freq['cumsum_min'] = tmaxminwei_freq.perc_min.cumsum()
tmaxminwei_freq = tmaxminwei_freq[['bin','pa_max','perc_max','cumsum_max','pa_min','perc_min','cumsum_min']]
#tmaxminwei_freq.append(tmaxminwei_freq.sum(numeric_only=True), ignore_index=True) # summary table

tmaxminwei_freq.loc["Row_Total"] = tmaxminwei_freq.sum()
tmaxminwei_freq

,bin,pa_max,perc_max,cumsum_max,pa_min,perc_min,cumsum_min
0,"(0.0, 0.1]",0.0,NaN,NaN,0.0,NaN,NaN
1,"(0.1, 0.2]",0.0,NaN,NaN,0.0,NaN,NaN
2,"(0.2, 0.3]",0.0,NaN,NaN,0.0,NaN,NaN
3,"(0.3, 0.4]",0.0,NaN,NaN,0.0,NaN,NaN
4,"(0.4, 0.5]",0.0,NaN,NaN,0.0,NaN,NaN
5,"(0.5, 0.6]",0.0,NaN,NaN,0.0,NaN,NaN
6,"(0.6, 0.7]",0.0,NaN,NaN,0.0,NaN,NaN
7,"(0.7, 0.8]",0.0,NaN,NaN,0.0,NaN,NaN
8,"(0.8, 0.9]",0.0,NaN,NaN,0.0,NaN,NaN
9,"(0.9, 1.0]",0.0,NaN,NaN,0.0,NaN,NaN
